# Joins com DuckDB

* [FROM and JOIN Clauses](https://duckdb.org/docs/stable/sql/query_syntax/from)
* [NULL Values](https://duckdb.org/docs/stable/sql/data_types/nulls)

In [1]:
import duckdb

In [2]:
conn = duckdb.connect()

In [3]:
conn.execute("""
    CREATE OR REPLACE TABLE pessoas (
        id INTEGER,
        nome TEXT,
        idade INTEGER,
        cidade_id INTEGER
    );
""")

conn.execute("""
    CREATE OR REPLACE TABLE cidades (
        id INTEGER,
        nome TEXT,
        estado TEXT
    );
""")

In [4]:
conn.execute("""
    INSERT INTO pessoas VALUES
        (1, 'Ana', 22, 1),
        (2, 'Bruno', 35, 2),
        (3, 'Carla', 19, 3),
        (4, 'Daniel', 40, 2),
        (5, 'Eduarda', 28, NULL);
""")

conn.execute("""
    INSERT INTO cidades VALUES
        (1, 'Rio de Janeiro', 'RJ'),
        (2, 'São Paulo', 'SP'),
        (3, 'Belo Horizonte', 'MG'),
        (4, 'Curitiba', 'PR'),
        (5, 'Porto Alegre', 'RS');
""")

# INNER JOIN
retorna linhas onde a condição de junção é satisfeita em ambas as tabelas.

In [5]:
display(conn.execute("""
    SELECT
        p.nome,
        p.idade,
        c.nome   AS cidade,
        c.estado
    FROM pessoas p
    INNER JOIN cidades c
        ON p.cidade_id = c.id;
""").df())

,nome,idade,cidade,estado
0,Ana,22,Rio de Janeiro,RJ
1,Daniel,40,São Paulo,SP
2,Carla,19,Belo Horizonte,MG
3,Bruno,35,São Paulo,SP


# LEFT JOIN
Retorna todas as linhas da tabela à esquerda e as correspondentes da tabela à direita. Se não houver correspondência, os valores da tabela da direita serão NULL.

In [6]:
display(conn.execute("""
    SELECT
        p.nome,
        p.idade,
        c.nome   AS cidade,
        c.estado
    FROM pessoas p
    LEFT JOIN cidades c
        ON p.cidade_id = c.id;
""").df())

,nome,idade,cidade,estado
0,Ana,22,Rio de Janeiro,RJ
1,Daniel,40,São Paulo,SP
2,Carla,19,Belo Horizonte,MG
3,Bruno,35,São Paulo,SP
4,Eduarda,28,None,None


# RIGHT JOIN
Retorna todas as linhas da tabela à direita e as correspondentes da tabela à esquerda. Onde não houver correspondência, os valores da tabela da esquerda serão NULL.

In [7]:
display(conn.execute("""
    SELECT
        p.nome,
        p.idade,
        c.nome AS cidade,
        c.estado
    FROM pessoas p
    RIGHT JOIN cidades c
        ON p.cidade_id = c.id;
""").df())

,nome,idade,cidade,estado
0,Ana,22,Rio de Janeiro,RJ
1,Daniel,40,São Paulo,SP
2,Carla,19,Belo Horizonte,MG
3,Bruno,35,São Paulo,SP
4,None,<NA>,Curitiba,PR
5,None,<NA>,Porto Alegre,RS


# FULL OUTER JOIN
Retorna todas as linhas de ambas as tabelas. Onde não houver correspondência, os campos ausentes são preenchidos com NULL.

In [8]:
display(conn.execute("""
    SELECT
        p.nome,
        p.idade,
        c.nome   AS cidade,
        c.estado
    FROM pessoas p
    FULL OUTER JOIN cidades c
        ON p.cidade_id = c.id;
""").df())

,nome,idade,cidade,estado
0,Ana,22,Rio de Janeiro,RJ
1,Daniel,40,São Paulo,SP
2,Carla,19,Belo Horizonte,MG
3,Bruno,35,São Paulo,SP
4,None,<NA>,Curitiba,PR
5,None,<NA>,Porto Alegre,RS
6,Eduarda,28,None,None


# Produto cartesiano (CROSS JOIN ou JOIN sem ON)
O produto cartesiano retorna todas as combinações possíveis entre os registros das duas tabelas.

Se pessoas tem 5 linhas e cidades tem 5 linhas, o resultado terá 25 linhas (5 × 5).

In [9]:
display(conn.execute("""
    SELECT
        p.nome AS pessoa,
        c.nome AS cidade
    FROM pessoas p, cidades c;
""").df())

,pessoa,cidade
0,Ana,Rio de Janeiro
1,Bruno,Rio de Janeiro
2,Carla,Rio de Janeiro
3,Daniel,Rio de Janeiro
4,Eduarda,Rio de Janeiro
5,Ana,São Paulo
6,Bruno,São Paulo
7,Carla,São Paulo
8,Daniel,São Paulo
9,Eduarda,São Paulo


In [11]:
display(conn.execute("""
    SELECT
        p.nome AS pessoa,
        c.nome AS cidade
    FROM pessoas p
    CROSS JOIN cidades c;
""").df())

,pessoa,cidade
0,Ana,Rio de Janeiro
1,Ana,São Paulo
2,Ana,Belo Horizonte
3,Ana,Curitiba
4,Ana,Porto Alegre
5,Ana,São Paulo (Duplicata)
6,Bruno,Rio de Janeiro
7,Bruno,São Paulo
8,Bruno,Belo Horizonte
9,Bruno,Curitiba


# Produto cartesiano parcial com chaves duplicadas na tabela da direita
Quando há duplicatas na chave da tabela da direita, o JOIN cria um produto cartesiano parcial entre os registros com o mesmo valor.

In [12]:
# Adiciona duplicata na tabela cidades (cidade_id = 2)
conn.execute("""
    INSERT INTO cidades VALUES
        (2, 'São Paulo (Duplicata)', 'SP');
""")


In [13]:
display(conn.execute("""
    SELECT
        p.nome AS pessoa,
        c.nome AS cidade
    FROM pessoas p
    INNER JOIN cidades c ON p.cidade_id = c.id
""").df())

,pessoa,cidade
0,Ana,Rio de Janeiro
1,Daniel,São Paulo
2,Carla,Belo Horizonte
3,Daniel,São Paulo (Duplicata)
4,Daniel,São Paulo (Duplicata)
5,Bruno,São Paulo
6,Bruno,São Paulo (Duplicata)
7,Bruno,São Paulo (Duplicata)


In [14]:
conn.close()